# Midterm 2

## FINM 36700 - 2024

### Brian Wickman

## Data

**All data files are found in the class github repo, in the `data` folder.**

This exam makes use of the following data files:
* `midterm_2_data.xlsx`

This file contains the following sheets:
- for Section 2:
    * `sector stocks excess returns` - MONTHLY excess returns for 49 sector stocks
    * `factors excess returns` - MONTHLY excess returns of AQR factor model from Homework 5
- for Section 3:
    * `factors excess returns` - MONTHLY excess returns of AQR factor model from Homework 5

In [2]:
# Set-up & Import data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")
sns.set_theme()

# import helper functions
import importlib
import helper_funcs

# Import data
FILEIN = 'data/midterm_2_data.xlsx'

factors = pd.read_excel(FILEIN, sheet_name='factors excess returns').set_index('date')
sectors = pd.read_excel(FILEIN, sheet_name='sector excess returns').set_index('date')

# 1. Short Answer

### 1.1.

Historically, which pricing factor among the ones we studied has shown a considerable decrease in importance?

From our analysis with tangency portfolios, the SMB factor has shown a considerable decrease in importance. From a time-series graph of cumulative returns perspective, (1) momentum has suffered since 2009 and (2) value (HML) turned into a discount (negative) in the recent past.

### 1.2.

True or False: For a given factor model and a set of test assets, the addition of one more factor to that model will surely decrease the cross-sectional MAE. 

True or False: For a given factor model and a set of test assets, the addition of one more factor to that model will surely decrease the time-series MAE. 

Along with stating T/F, explain your reasoning for the two statements.

1) False, the new factor could be perfectly (0.999) correlated with anothe factor and not improve the regression fit nor cross-sectional MAE. The cross-sectional MAE (>0) suggests that there exists risk-free excess returns (factor model not capturing all risk premia). Thus, adding a highly correlated factor might not improve cross-sectional fit.
2) False, no obvious reason why an additional covariate to a linear regression will lower the MAE of the intercepts (i.e. shrink the intercepts toward zero). 

### 1.3.

Consider the scenario in which you are helping two people with investments.

* The young person has a 50 year investment horizon.
* The elderly person has a 10 year investment horizon.
* Both individuals have the same portfolio holdings.

State who has the more certain cumulative return and explain your reasoning.

If log returns are assumed to be IID and normally distributed, the 50-year investment horizon investor has a more certain cumulative return from a statistical perspective. We know that mean annualized return converges to true annual mean return as investment horizon gets large and the variance of annualized returns go to zero, which favors the longer investment horizon. From a historical perspective, if both individuals have broad market exposure, past performance suggests longer investment horizons would win as returns compound, if markets keep increasing. However, if these two individuals are 100% in Bitcoin (reached new high today), the investment could crash eleven years from now and sabotage the young person's 50-year investment horizon. In short, the longer-horizon has a higher probability of cumulative return, although it is not guaranteed.

### 1.4.

Suppose we find that the 10-year bond yield works well as a new pricing factor, along with `MKT`.

Consider two ways of building this new factor.
1. Directly use the index of 10-year yields, `YLD`
1. Construct a Fama-French style portfolio of equities, `FFYLD`. (Rank all the stocks by their correlation to bond yield changes, and go long the highest ranked and shor tthe lowest ranked.)

Could you test the model with `YLD` and the model with `FFYLD` in the exact same ways? Explain

Initial thought: YLD is not necessarily a bad idea because would not be correlated with MKT factor as interest rates rely on business cycles (so maybe a little correlated lol).
No, the index of 10-year yields would not be returns, but rather yields, a more economic variable. Typically, researchers would use changes in yields rather than yield levels and maybe even consider a non-linear time-series model as the yield curve has unique statistical properties.
**I looked this question up on COPILOT.**


### 1.5.

Suppose we implement a momentum strategy on cryptocurrencies rather than US stocks.

Conceptually speaking, but specific to the context of our course discussion, how would the risk profile differ from the momentum strategy of US equities?

The crypto momentum strategy would be much riskier (higher volatility) because the universe of reliable (read: non-fraudelent) cryptocurrencies is small and the momentum strategy relies on large universe of assets (that are also liquid)/ from which to pick big winners and losers. Without the distinction between big winners and losers, the small edge of momentum is not viable. Furthermore, I would be interested in analyzing the autocorrelation of crypto returns before deploying this strategy.

***

# 2. Pricing and Tangency Portfolio

You work in a hedge fund that believes that the AQR 4-Factor Model (present in Homework 5) is the perfect pricing model for stocks.

$$
\mathbb{E} \left[ \tilde{r}^i \right] = \beta^{i,\text{MKT}} \mathbb{E} \left[ \tilde{f}_{\text{MKT}} \right] + \beta^{i,\text{HML}} \mathbb{E} \left[ \tilde{f}_{\text{HML}} \right] + \beta^{i,\text{RMW}} \mathbb{E} \left[ \tilde{f}_{\text{RMW}} \right] + \beta^{i,\text{UMD}} \mathbb{E} \left[ \tilde{f}_{\text{UMD}} \right]
$$

The factors are available in the sheet `factors excess returns`.

The hedge fund invests in sector-tracking ETFs available in the sheet `sectors excess returns`. You are to allocate into these sectors according to a mean-variance optimization with...

* regularization: elements outside the diagonal covariance matrix divided by 2.
* modeled risk premia: expected excess returns given by the factor model rather than just using the historic sample averages.

You are to train the portfolio and test out-of-sample. The timeframes should be:
* Training timeframe: Jan-2018 to Dec-2022.
* Testing timeframe: Jan-2023 to most recent observation.

In [3]:
# Set-up
train_start = '2018-01-01'
train_end = '2022-12-31'
test_start = '2023-01-01'

### 2.1.
(8pts)

Calculate the model-implied expected excess returns of every asset.

The time-series estimations should...
* NOT include an intercept. (You assume the model holds perfectly.)
* use data from the `training` timeframe.

With the time-series estimates, use the `training` timeframe's sample average of the factors as the factor premia. Together, this will give you the model-implied risk premia, which we label as
$$
\lambda_i := \mathbb{E}[\tilde{r}_i]
$$

* Store $\lambda_i$ and $\boldsymbol{\beta}^i$ for each asset.
* Print $\lambda_i$ for `Agric`, `Food`, `Soda`

In [59]:
# CAPM RP Estimation (NO INTERCEPT)
importlib.reload(helper_funcs) # no intercept, no alpha to report
train_sectors = sectors.loc[train_start:train_end]
train_factors = factors.loc[train_start:train_end]

# Time-series estimation (Beta^i's), without intercept
capm_ts_results = pd.DataFrame()
for col in train_sectors.columns:
    # Process the column
    temp_df = helper_funcs.capm_ts_regression(train_sectors[col], factor = train_factors)
    # Concatenate the result to the final DataFrame
    capm_ts_results = pd.concat([capm_ts_results, temp_df])
betas = capm_ts_results.copy()
lambdas = ((train_factors.mean() * 12) * betas[['MKT', 'HML', 'RMW', 'UMD']]).sum(axis = 1).to_frame('lambda_i')
display(lambdas.loc[['Agric', 'Food ', 'Soda ']]) # spaces are hard :/

# Construct time-series of excess returns based on factor model
train_expected_excess_returns = pd.DataFrame(index=train_sectors.index)
for portfolio in betas.index:
    # Multiply the weights by the time-series values (how much exposure to each factor * factor)
    portfolio_returns = factors.mul(betas.loc[portfolio], axis=1)

    # Sum the results to get the monthly portfolio returns
    train_expected_excess_returns[portfolio] = portfolio_returns.sum(axis=1)

# Print the resulting DataFrame
display(train_expected_excess_returns.head(5))


,lambda_i
Agric,0.043861
Food,0.065451
Soda,0.088035


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0.044583,0.024118,0.023565,0.023490,0.013633,0.092526,0.082981,0.042938,0.018345,0.054146,...,0.035283,0.054598,0.048146,0.063889,0.045164,0.045476,0.041157,0.047674,0.053868,0.043315
2018-02-01,-0.037196,-0.019921,-0.018031,-0.018412,-0.025619,-0.050492,-0.054147,-0.058826,-0.015086,-0.045314,...,-0.031793,-0.047034,-0.039448,-0.030036,-0.033692,-0.053357,-0.029675,-0.071131,-0.044868,-0.035987
2018-03-01,-0.019005,-0.013997,-0.015890,-0.014952,-0.012768,-0.029957,-0.027568,-0.019525,-0.013700,-0.025312,...,-0.017923,-0.024858,-0.024156,-0.024395,-0.022258,-0.022766,-0.019953,-0.022981,-0.024363,-0.020979
2018-04-01,0.017814,-0.004940,-0.012993,-0.013821,-0.006473,0.028622,0.023893,0.013174,-0.020010,-0.006784,...,-0.005177,0.003678,-0.001355,-0.000929,-0.004184,0.012053,0.002052,0.013410,0.006380,0.003821
2018-05-01,0.015938,0.000797,-0.004010,-0.000873,-0.014497,0.066757,0.057953,0.012131,-0.004583,0.020362,...,0.008812,0.019617,0.011415,0.040149,0.012918,0.006787,0.011159,0.009493,0.019112,0.011240


### 2.2.

Use the expected excess returns derived from (2.1) with the **regularized** covariance matrix to calculate the weights of the tangency portfolio.

- Use the covariance matrix only for `training` timeframe.
- Calculate and store the vector of weights for all the assets.
- Return the weights of the tangency portfolio for `Agric`, `Food`, `Soda`.

$$
\textbf{w}_{t} = \dfrac{\tilde{\Sigma}^{-1} \bm{\lambda}}{\bm{1}' \tilde{\Sigma}^{-1} \bm{\lambda}}
$$

Where $\tilde{\Sigma}^{-1}$ is the regularized covariance-matrix.

In [61]:
# tangency weights use expected excess returns, not 
wts = helper_funcs.tangency_weights(train_expected_excess_returns, cov_mat = 0.5)
display(wts.loc[['Agric', 'Food ', 'Soda ']])

,Tangent Weights
Agric,-0.030451
Food,0.063498
Soda,0.104376


### 2.3.

Evaluate the performance of this allocation in the `testing` period. Report the **annualized**
- mean
- vol
- Sharpe

In [62]:
# performance of regularized MV optimization
# using testing data, not estimated excess returns
test_sectors = sectors.loc[test_start:]
reg_performance = helper_funcs.summary_stats(test_sectors@wts)
display(reg_performance)

,Mean,Volatility,Sharpe,VaR (0.05)
Tangent Weights,0.0967,0.2055,0.4706,-0.0766


### 2.4.

(7pts)

Construct the same tangency portfolio as in `2.2` but with one change:
* replace the risk premia of the assets, (denoted $\lambda_i$) with the sample averages of the excess returns from the `training` set.

So instead of using $\lambda_i$ suggested by the factor model (as in `2.1-2.3`) you're using sample averages for $\lambda_i$.

- Return the weights of the tangency portfolio for `Agric`, `Food`, `Soda`.

Evaluate the performance of this allocation in the `testing` period. Report the **annualized**
- mean
- vol
- Sharpe

In [65]:
# risk premia based on sample averages
lambdas_hist = train_sectors.mean()
# Construct time-series of excess returns based on factor model
train_expected_excess_returns2 = pd.DataFrame(index=train_sectors.index)
for portfolio in lambdas_hist.index:
    # Multiply the historical RP by the time-series values
    # how much exposure to each factor
    portfolio_returns = factors.mul(lambdas_hist.loc[portfolio], axis=1)

    # Sum the results to get the monthly portfolio returns
    train_expected_excess_returns2[portfolio] = portfolio_returns.sum(axis=1)

# Calculate tangency weights
wts2 = helper_funcs.tangency_weights(train_expected_excess_returns2, cov_mat = 0.5)
display(wts2.loc[['Agric', 'Food ', 'Soda ']])

# tangency portfolio performance
reg_performance2 = helper_funcs.summary_stats(test_sectors@wts2)
display(reg_performance2)

,Tangent Weights
Agric,0.010436
Food,0.023200
Soda,0.011587


,Mean,Volatility,Sharpe,VaR (0.05)
Tangent Weights,0.1088,0.1634,0.6659,-0.0535


### 2.5.

Which allocation performed better in the `testing` period: the allocation based on premia from the factor model or from the sample averages?

Why might this be?

In [67]:
perf_comp = pd.concat([reg_performance, reg_performance2])
perf_comp.index = ['TS_Premia', 'Hist_Premia']
display(perf_comp)

,Mean,Volatility,Sharpe,VaR (0.05)
TS_Premia,0.0967,0.2055,0.4706,-0.0766
Hist_Premia,0.1088,0.1634,0.6659,-0.0535


Interestingly enough, the premia estimated from historical averages performed better, with a higher mean return, Sharpe Ratio, and smaller VaR. This implies that the modeled risk premia were not robust to out-of-sample testing and that the factors performed close to their historical averages.

### 2.6.
Suppose you now want to build a tangency portfolio solely from the factors, without using the sector ETFs.

- Calculate the weights of the tangency portfolio using `training` data for the factors.
- Again, regularize the covariance matrix of factor returns by dividing off-diagonal elements by 2.

Report, in the `testing` period, the factor-based tangency stats **annualized**...
- mean
- vol
- Sharpe


In [70]:
# Calculate tangency weights
wts_fct = helper_funcs.tangency_weights(train_factors, cov_mat = 0.5)
display(wts_fct)

# tangency portfolio performance
test_factors = factors.loc[test_start:]
reg_performance3 = helper_funcs.summary_stats(test_factors@wts_fct)
display(reg_performance3)

,Tangent Weights
MKT,0.176921
HML,-0.016221
RMW,0.598427
UMD,0.240872


,Mean,Volatility,Sharpe,VaR (0.05)
Tangent Weights,0.0624,0.0582,1.0722,-0.0244


### 2.7.

Based on the hedge fund's beliefs, would you prefer to use the ETF-based tangency or the factor-based tangency portfolio? Explain your reasoning. Note that you should answer based on broad principles and not on the particular estimation results.

I would prefer to use the ETF-based tangency because this approach uses a wide variety of test assets to capture systematic risk, whereas the factor-based tangency portfolio uses four factors, albeit they have low correlation. Additionally, the factor based tangency portfolio cannot be traded (unless using replica univariate factor ETFs) whereas the ETFs are tradeable assets.

***

# 3. Long-Run Returns

For this question, use only the sheet `factors excess returns`.

Suppose we want to measure the long run returns of various pricing factors.

### 3.1.

Turn the data into log returns.
- Display the first 5 rows of the data.

Using these log returns, report the **annualized**
* mean
* vol
* Sharpe

### 3.2.

Consider 15-year cumulative log excess returns. Following the assumptions and modeling of Lecture 6, report the following 15-year stats:
- mean
- vol
- Sharpe

How do they compare to the estimated stats (1-year horizon) in `3.1`? 

In [35]:
# factor log returns
fact_log_ret = factors.apply(lambda x: np.log(1 + x))
print("Log returns of factors:")
display(fact_log_ret.head(5))

# summary stats annualized
print("Log return summary stats:")
display(helper_funcs.summary_stats(fact_log_ret).iloc[:,0:3])

# 15-year rolling cumulative returns
rolling_window = 15 * 12  # 15 years * 12 months
cum_log_ret = fact_log_ret.rolling(window=rolling_window).sum()
print("15-year cumulative log return")
display(helper_funcs.summary_stats(cum_log_ret).iloc[:,0:3])

Log returns of factors:


,MKT,HML,RMW,UMD
date,,,,
1980-01-01,0.053636,0.017349,-0.017146,0.072786
1980-02-01,-0.012275,0.006081,0.000400,0.075849
1980-03-01,-0.138113,-0.010151,0.014494,-0.100373
1980-04-01,0.038932,0.010544,-0.021224,-0.004309
1980-05-01,0.051263,0.003793,0.003394,-0.011263


Log return summary stats:


,Mean,Volatility,Sharpe
MKT,0.0735,0.1588,0.4628
HML,0.0198,0.1098,0.1803
RMW,0.0435,0.0836,0.5203
UMD,0.0501,0.1604,0.3123


15-year cumulative log return


,Mean,Volatility,Sharpe
MKT,11.9712,1.3738,8.7139
HML,3.7910,1.5158,2.5010
RMW,7.4191,0.5232,14.1802
UMD,8.8978,2.5285,3.5190


Compared to the 1-year log returns, the 15-year cumulative log returns are slighly more volatile but with better Sharpe ratios, which makes sense as the variance and SR of cumulative returns grows as a function of the horizon. The mean returns are much larger than the one-period returns, by more than 15x times, which demonstrates the power of compounding returns (even though means scale linearly with horizon).

### 3.3.

What is the probability that momentum factor has a negative mean excess return over the next 
* single period?
* 15 years?

In [39]:
# Probability that '0' will outperform UMD in next 1 month, 15 years
spread = - fact_log_ret['UMD']
mu, sigma = spread.mean() * 12, spread.std() * np.sqrt(12)
prob_1m = norm.cdf(np.sqrt(1) * mu / sigma)
prob_15y = norm.cdf(np.sqrt(15 * 12) * mu / sigma)
print(f'Probability that UMD will have negative mean excess return in\n'
      f'1-month: {round(prob_1m, 4)}\n'
      f'15-years: {round(prob_15y, 4)}')

Probability that UMD will have negative mean excess return in
1-month: 0.3774
15-years: 0.0


### 3.4.

Recall from the case that momentum has been underperforming since 2009. 

Using data from 2009 to present, what is the probability that momentum *outperforms* the market factor over the next
* period?
* 15 years?

In [40]:
# Probability that UMD will outperform MKT in next 1 month, 15 years
spread = fact_log_ret['UMD'] - fact_log_ret['MKT']
mu, sigma = spread.mean() * 12, spread.std() * np.sqrt(12)
prob_1m = norm.cdf(np.sqrt(1) * mu / sigma)
prob_15y = norm.cdf(np.sqrt(15*12) * mu / sigma)
print(f'Probability that UMD will outperform MKT in\n'
      f'1-month: {round(prob_1m,4)}\n'
      f'15-years: {round(prob_15y,4)}')

Probability that UMD will outperform MKT in
1-month: 0.4618
15-years: 0.099


### 3.5.
Conceptually, why is there such a discrepancy between this probability for 1 period vs. 15 years?

What assumption about the log-returns are we making when we use this technique to estimate underperformance?

We assume that log-returns are IID and normally-distributed in order to apply the CLT and use the standard normal CDF. There is a discrepancy between the 1-month and 15-year probability because at a short time horizon, 'anything' can happen as the series have considerable variances. However, over the long run (15 years), the series converge to their true annual mean return, which we are implicitly assuming is their historical mean.

### 3.6.

Using your previous answers, explain what is meant by time diversification.

Time diversification is the idea that mean annualized return becomes riskless for large investment horizons, as investments converge to their true annual mean return in the long run.

### 3.7.

Is the probability that `HML` and `UMD` both have negative cumulative returns over the next year higher or lower than the probability that `HML` and `MKT` both have negative cumulative returns over the next year?

Answer conceptually, but specifically. (No need to calculate the specific probabilities.)

This question asks if UMD or MKT are more likely to have negative cumulative reutrns over the next year. UMD is more likely to because it is **slightly** closer to zero in the z-score sense, as it has a smaller mean and a comparable volatility to MKT (see 3.1). Once again, this is a small investment horizon so anything could happen.

***